#Add Rapids Dataset! And GPU On!

![](https://image.slidesharecdn.com/jhhgtceurope2018final-181010152435/95/open-source-rapids-gpu-platform-to-accelerate-predictive-data-analytics-6-638.jpg?cb=1539186408)slideshare.net

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Columns Information

ALS Functional Rating Score (ALSFRS)

DDX11L1, WASP Family Homolog 7, Pseudogene), MIR6859-1 (MicroRNA 6859-1) is an RNA Gene.

Columns (53861) are Gene Names

<h1 style="background-color:#DC143C; font-family:'Brush Script MT',cursive;color:white;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">Predicting functional decline and survival in amyotrophic lateral sclerosis</h1>

Authors: Mei-Lyn Ong,, Pei Fang Tan, and Joanna D. Holbrook  Editor: Antonio Musaro

Published online 2017 Apr 13. doi: 10.1371/journal.pone.0174925

"In that study, time series data from PRO-ACT subjects were fitted to exponential models. Binary classes for decline in the total score of amyotrophic lateral sclerosis functional rating scale revised (ALSFRS-R) (fast/slow progression) and survival (high/low death risk) were derived. Data was segregated into training and test sets via cross validation. Learning algorithms were applied to the demographic, clinical and laboratory parameters in the training set to predict ALSFRS-R decline and the derived fast/slow progression and high/low death risk categories. The performance of predictive models was assessed by cross-validation in the test set using Receiver Operator Curves and root mean squared errors." 

"A model created using a boosting algorithm containing the decline in four parameters (weight, alkaline phosphatase, albumin and creatine kinase) post baseline, was able to predict functional decline class (fast or slow) with fair accuracy (AUC = 0.82). However similar approaches to build a predictive model for decline class by baseline subject characteristics were not successful. In contrast, baseline values of total bilirubin, gamma glutamyltransferase, urine specific gravity and ALSFRS-R item score—climbing stairs were sufficient to predict survival class."

"Using combinations of small numbers of variables it was possible to predict classes of functional decline and survival across the 1–2 year timeframe available in PRO-ACT. These findings may have utility for design of future ALS clinical trials."

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5390993/

In [ ]:
%%time
# Import the Rapids suite here - takes abot 2 mins

import sys
!cp ../input/rapids/rapids.0.17.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import gc
import sys
import random
from tqdm.notebook import * 
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from cuml.linear_model import LogisticRegression
from cuml.svm import SVC, SVR
from cuml.neighbors import KNeighborsClassifier, NearestNeighbors

from cuml.metrics import roc_auc_score
from cuml.preprocessing import LabelEncoder
from cuml.experimental.preprocessing import MinMaxScaler
from cuml.linear_model import MBSGDClassifier as cumlMBSGDClassifier
from cuml.naive_bayes import MultinomialNB

from sklearn.model_selection import StratifiedKFold

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source

import warnings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
warnings.simplefilter("ignore")
warnings.filterwarnings('ignore')

<font color="#EC7063">Machine learning algorithms</font>


"For each analysis, four algorithms: RUS/Ada Boost, Naïve Bayes, Decision Tree and Random Forest were run. The algorithms were chosen as capable of handling categorical outcomes and missing data in the variables."

"From the Boost suite of algorithms, RUS Boost was used for predicting fast and slow progressing patients as it is able to tolerate class imbalances; there are more subjects in the slow progression than fast progression category. This was not a factor for the high-low death risk, survival prediction because subjects are distributed more evenly so adaptive (Ada) Boost Decision Tree was used as it is less susceptible to over-fitting." 

"The best models from each algorithm were compared with each other. Those with the higher area from a receiver operating characteristic curve (ROC) curve and lower nRMSE were declared the best performing model. In most cases this was the Boost algorithms, excepting the case of individualized ALSFRS-R k from baseline, for which the Random Forests model was reported."

"To avoid modeling artifacts due to noise in the data, the 1,568 subjects with ALSFRS-R data were categorized into two groups using partitioning around medoids (PAM) and K-means clustering on their decline parameter (k). Both methods implied a binary classification was optimal. The k-means cluster categorisation was used for subsequent analyses since it showed a slightly higher median ALSFRS-R rate difference between the two groups than that obtained by PAM. They were denoted fast and slow progressing patients and there was an estimated median 20 point drop in ALSFRS-R score over a 6 month period (3.3 points /month) for the fast progressing compared to a 3 point drop (0.5 points / month) for the slow progressing patients."


Authors: Mei-Lyn Ong,, Pei Fang Tan, and Joanna D. Holbrook Editor: Antonio Musaro

Published online 2017 Apr 13. doi: 10.1371/journal.pone.0174925
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5390993/

"Subjects cluster into binary classes for progression and survival in PRO-ACT.
(A) Frequency histogram of the fitted trajectory of ALSFRS-R score (k) in the exponential model. Lines are coloured via the k-means of the same parameter into decline class, fast progressing = blue, slow progressing = red. (B) Mean values of fitted trajectory of ALSFRS-R score (k) in the exponential model over a 6 month time period for for the k-means clustering derived decline classes, fast progressing = blue, slow progressing = red (as in 1A). Dotted lines represent 95% confidence bounds. (C) Frequency histogram of the time to death or time of last follow-up. Lines are coloured via subject classification into survival classes (as opposed to the progression classes in 1A, B and D) derived from k-mean clustering of the number of days from the start of the trail to day of death or last follow-up contact date, high death risk = green, low death risk = orange. (D) Kaplan-Meier plot of estimated survival probability by decline class (again derived from k-means clustering of the fitted trajectory of ALS-FRS-R score (k) in the exponential model, akin to 1A and 1B) using univariate Cox regression. The fast progressing group (blue n = 218) has an increased death risk compared to the slow progressing group (red n = 572) (hazard ratio = 4.21, p<2x10-16.)"

![](https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0174925.g001&type=large)
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5390993/
journals.plos.org

In [ ]:
import cudf


df = cudf.read_csv("../input/end-als/end-als/transcriptomics-data/DESeq2/median_low_vs_high.csv")

print("Median Low vs High size:", df.shape)

In [ ]:
df.head()

"Baseline variables did not accurately predict ALSFRS-R decline class but were able to fairly accurately predict individual slope of decline with nRMSE = 15%"

"The authors attempted to use only the baseline variables (i.e. all 113 variables in the dataset at beginning of the study (t = 0). For time-series data this means the fitted A parameter representing estimated value at t = 0) to predict decline class. 

"They excluded ALSFRS-R subscales. However the best model (using RUS Boost) achieved only very poor prediction accuracy (74% of slow progressing patients were predicted as such but only 39% of actual fast progressing patient were predicted as such)."

"Surprisingly, when they performed random forest and Ada boosting using regression trees on all variables at baseline (excluding ALSFRS-R subscales) to predict individual slope of ALSFRS-R decline (k) as opposed to decline class they were able to achieve reasonable predictive power. Both models performed similarly and predicted ALSFRS-R decline with nRMSE = 15%. After variable selection, eight variables were sufficient to recapitulate the prediction accuracy of the random forests model and nine variables were sufficient in the Ada Boost model. Weight, pulse and blood pressure were common to both models."

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5390993/

In [ ]:
df.info()

In [ ]:
print(f'We have {df.Participant_ID.nunique()} unique participants in our set')

In [ ]:
DATA_PATH = "../input/end-als/end-als/transcriptomics-data/DESeq2/"

In [ ]:
os.listdir(DATA_PATH)

<font color="#EC7063">Survival class was also fairly accurately (AUC = 0.8) predicted using four variables measured at baseline</font>



"When all the variables available to the machine learning algorithms were restricted to fitted values at baseline only, the Ada Boost algorithm predicted survival class with only slightly reduced accuracy (72% and 75% of high death risk and low death risk subjects were correctly classified, respectively." 

"The AUC was calculated to be 0.77. After variable selection, the baseline values of total bilirubin, gamma glutamyltransferase, urine specific gravity and ALSFRS-R climbing stairs item score together resulted accuracy similar to the full model. Subjects in the high death risk group tended to have lower total bilirubin at baseline and higher values for each of the following parameters: urine specific gravity, climbing stairs and gamma glutamyltransferase values at baseline, compared to subjects in the low death risk group."

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5390993/

In [ ]:
!pip install scikit-allel

![](https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0174925.g005&type=large)journals.plos.org
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5390993/

<font color="#EC7063">Survival class is predicted from baseline variables.</font>


(A) Prediction accuracy by receiver operating curve (ROC) for model-defined survival class (includes subjects with last date of follow-up used as a proxy for date of death) using baseline for all variables in Ada-Boost model. (B-E) Fitted baseline values showing mean, 95% confidence interval and one standard deviation for (B) bilirubin, (C) urine specific gravity, (D) climbing stairs, (E) gamma glutamyltransferase (y-axis) for model-defined high death risk group (green) and low death risk group (orange).

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5390993/

In [ ]:
from sklearn.model_selection import train_test_split # machine learning
from sklearn.ensemble import RandomForestClassifier # machine learning
import shap # model explainability
import eli5 # model explainability
import allel # work with genetic data

In [ ]:
example_low_high = '/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/median_low_vs_high.csv'

In [ ]:
#Code by Paul Mooney  https://www.kaggle.com/paultimothymooney/starter-notebook-for-end-als-kaggle-challenge

median_low_vs_high = pd.read_csv(example_low_high)
median_low_vs_high.to_csv('/kaggle/working/median_low_vs_high.csv')

sns.histplot(median_low_vs_high.ALSFRS_Class_Median);

In [ ]:
#Visualize some of the original dimensions

import plotly.express as px

features = ["FAM138A", "DDX11L1", "WASH7P", "MIR1302.2HG", "ENSG00000278858"]

fig = px.scatter_matrix(
    median_low_vs_high,
    dimensions=features,
    color="ALSFRS_Class_Median"
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
plt.figure(1, figsize=(5,5))
median_low_vs_high['ALSFRS_Class_Median'].value_counts().plot.pie(autopct="%1.1f%%")

In [ ]:
#Code by Puru Behl https://www.kaggle.com/accountstatus/mt-cars-data-analysis

sns.distplot(median_low_vs_high['ALSFRS_Class_Median'])
plt.axvline(median_low_vs_high['ALSFRS_Class_Median'].values.mean(), color='red', linestyle='dashed', linewidth=1)
plt.title('ALSFRS Class Median distribution')

In [ ]:
#Code by Firat Gonen https://www.kaggle.com/frtgnn/elo-eda-lgbm/notebook 

plt.figure(figsize=(10, 6))
plt.title('WASH7P Distribution')
sns.despine()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

sns.distplot(median_low_vs_high['WASH7P'], hist=True, rug=False,norm_hist=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

#fill in mean for floats
for c in median_low_vs_high.columns:
    if median_low_vs_high[c].dtype=='float16' or  median_low_vs_high[c].dtype=='float32' or  median_low_vs_high[c].dtype=='float64':
        median_low_vs_high[c].fillna(df[c].mean())

#fill in -999 for categoricals
median_low_vs_high = median_low_vs_high.fillna(-999)
# Label Encoding
for f in median_low_vs_high.columns:
    if median_low_vs_high[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(median_low_vs_high[f].values))
        median_low_vs_high[f] = lbl.transform(list(median_low_vs_high[f].values))
        
print('Labelling done.')

In [ ]:
median_low_vs_high = pd.get_dummies(median_low_vs_high)

In [ ]:
x = median_low_vs_high.drop(['Participant_ID', 'ALSFRS_Class_Median'], axis=1)
x.fillna(999999, inplace=True)
y = median_low_vs_high['ALSFRS_Class_Median']

In [ ]:
dt = DecisionTreeClassifier(max_depth=3)

In [ ]:
dt.fit(x, y)

In [ ]:
dt_feat = pd.DataFrame(dt.feature_importances_, index=x.columns, columns=['feat_importance'])
dt_feat.sort_values('feat_importance').tail(8).plot.barh(figsize=(12, 6), color='r')
plt.show()

In [ ]:
from IPython.display import SVG
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

graph = Source(export_graphviz(dt, out_file=None, feature_names=x.columns, filled = True))
display(SVG(graph.pipe(format='svg')))

Probably that Code is Completely Wrong. There are 53861 Columns, it's very hard for a beginner even to understand what's going on. I hope to learn with others work. My GPU quota is being consumed and I'm doing Nothing else.